### Notes
Diese Implementierung basiert direkt auf **Attilio Meuccis Artikel aus dem Jahr 2008** „The Black-Litterman Approach: Original Model and Extensions” (SSRN 1117574). Jede Formel wurde sorgfältig gemäß der Notation und den Ableitungen des Artikels implementiert.

## Mathematische Grundlagen

### Wie die Portfolio Optimierung mit dem Black-Litterman Modell gelöste wird
Die traditionelle Portfoliooptimierung auf Basis historischer erwarteter Renditen weist folgende Nachteile auf:
- **Schätzungsrisiko**: Stichprobenmittelwerte sind äußerst ungenaue Schätzwerte
- **Extreme Allokationen**: Geringfügige Änderungen der Eingaben, daraus folgt stark unterschiedliche Portfolios
- **Unfähigkeit, Meinungen einzubeziehen**: Keine natürliche Möglichkeit, Marktdaten mit subjektiven Meinungen zu kombinieren


Black-Litterman bietet einen Bayes'schen Rahmen, um:
1. von einer **stabilen Prior-Verteilung** (oft Gleichgewicht) auszugehen,
2. **die Ansichten der Anleger** mit angemessenen Konfidenzniveaus zu berücksichtigen,
3. **reibungslose, intuitive Allokationen** zu erzielen.

Die Prior-Verteilung

Die Marktrenditen werden als normalverteilt modelliert:

$$
X ~ N(π, Σ)
$$

Wobei:
- **$X$**: Zufallsvektor der Vermögensrenditen (N × 1)
- **$\pi$**: Vorher erwartete Renditen (N × 1)
- **$\Sigma$**: Kovarianzmatrix (N × N)

Berechnung von π aus dem Gleichgewicht (CAPM)

Die erwarteten Gleichgewichtsrenditen werden aus den Marktgewichten rückberechnet:

$$
\pi = 2 \lambda \Sigma w_{\text{eq}}
$$

Dabei gilt:
- **$w_{eq}$**: Marktkapitalisierungsgewichte (N × 1)
- **$Λ$** (Lambda): Risikoaversion-Parameter (typischerweise 2-3)
- Dies ergibt sich aus der FOC der Mean-Variance-Optimierung: w* = argmax{w'π - λw'Σw}

**Implementierung:**
```python
pi = BlackLittermanModel.compute_equilibrium_returns(w_eq, Sigma, risk_aversion=2.5)
```

Ansichten und Unsicherheit

Ansichten werden als lineare Kombinationen der erwarteten Renditen ausgedrückt:

$$
P \pi \approx N(Q, Ω)
$$

Dabei gilt:
- **P**: Auswahlmatrix (K × N), jede Zeile definiert ein Meinungsportfolio
- **Q**: Erwartete Renditen der Meinungen (K × 1)
- **$\Omega$**: Unsicherheitsmatrix der Meinungen (K × K)

 Erstellen der Auswahlmatrix P

**Absolute Sichtweise auf Vermögenswert i:**
```
P[k,i] = 1,  alle anderen Einträge = 0
```
Beispiel: „Tech-Aktien werden eine Rendite von 12 % erzielen”
```python
P = [0, 0, 1, 0]  # Sichtweise auf Vermögenswert 2
Q = [0.12]
```

**Relative Sichtweise (Vermögenswert i übertrifft Vermögenswert j):**
```
P[k,i] = 1,  P[k,j] = -1,  alle anderen Einträge = 0
```
Beispiel: „Technologie wird Energie um 5 % übertreffen”
```python
P = [1, -1, 0, 0]  # Technologie – Energie
Q = [0.05]
```

**Portfolio-Ansicht:**
```
P[k,:] = Portfolio-Gewichtungen
```
Beispiel: „Ein gleichgewichtetes Portfolio der ersten beiden Vermögenswerte erzielt eine Rendite von 8 %”
```python
P = [0,5, 0,5, 0, 0]
Q = [0.08]
```


Angabe der Unsicherheit $\Omega$

**Empfehlung von Meucci (Gleichung 12):**
$$
\Omega = (1/c) × P\Sigma P'
$$

Dabei gilt:
- **c**: Konfidenzparameter (höher = größere Konfidenz in den Ansichten)
- Die Struktur übernimmt Korrelationen aus dem Markt $\Sigma$

**Mit relativer Konfidenz (Gleichung 13):**
$$
\Omega = (1/c) × diag(u) × P \Sigma P' × diag(u)
$$

Wobei:
- **u**: Relativer Konfidenzvektor (K × 1)
- Höheres $μ[k]$ → GERINGERE Konfidenz in Ansicht k

**Implementierung:**
```python
# Automatische Omega-Berechnung
mu_bl, sigma_bl = bl.compute_posterior(P, Q, confidence=2.0)

# Manuelle Omega-Spezifikation
Omega = np.array([[0.001, 0], [0, 0.002]])  

# Unterschiedliche Zuverlässigkeit pro Ansicht
mu_bl, sigma_bl = bl.compute_posterior(P, Q, Omega=Omega)
```

### Posteriorverteilung

#### Ursprüngliche Formulierung (Abschnitt 2)

Die Ansichten beziehen sich auf den **Parameter** $\mu$:
$$
\mu | Q ; \Omega \verb|~| ̃N(\mu_{BL}, \Sigma_\mu)
$$

Die posterior Marktverteilung lautet:
$$
X |Q ; \Omega \verb|~| N(\mu_{BL}, \Sigma_{BL})
$$

**Wichtige Formeln (Gleichungen 20–21):**
$$
\mu_{BL} = \pi + \tau \Sigma P' ( \tau P \Sigma P' +  \Omega)^{-1} (Q-P\pi)
\\
\Sigma_{BL} = (1+\tau) \Sigma - \tau^2 \Sigma P' (\tau P \Sigma P' + \Omega)^{-1} P \Sigma
$$

Dabei ist τ die Unsicherheit in der Prior-Verteilung (typischerweise 0,01–0,05).

#### Marktformulierung (Abschnitt 3) **[EMPFOHLEN]**

Die Ansichten beziehen sich direkt auf die **Marktrenditen** X:
```
V|x ~ N(Px, Ω)
```

**Wichtige Formeln (Gleichungen 32-33):**
```
μ_BL^m = π + ΣP'(PΣP' + Ω)^(-1)(Q - Pπ)

Σ_BL^m = Σ - ΣP'(PΣP' + Ω)^(-1)PΣ
```

**Vorteile der Marktformulierung:**
1. **Kein τ-Parameter erforderlich** im Posterior
2. **Korrektes Grenzverhalten:**
   - Ω → ∞ (kein Vertrauen): Posterior = Prior ✓
   - Ω → 0 (volles Vertrauen): Posterior = bedingte Verteilung ✓
3. **Natürliche Integration der Szenarioanalyse**

**Implementierung:**
```python
bl = BlackLittermanModel(pi, Sigma, use_market_formulation=True)  # Empfohlen
mu_bl, sigma_bl = bl.compute_posterior(P, Q, confidence=2.0)
```

### Grenzfälle

#### Kein Vertrauen (Ω → ∞)

Die A-posteriori-Wahrscheinlichkeit sollte der A-priori-Wahrscheinlichkeit entsprechen:
```
X|Q → X  als Ω → ∞
```

**Marktformulierung:** ✓ Erreicht dies genau.
**Ursprüngliche Formulierung:** A-posteriori = N(π, (1+τ)Σ) ≠ A-priori genau.

**Test:**
```python
mu_bl, _ = bl.compute_posterior(P, Q, confidence=0.001)  # Sehr niedrig.
# mu_bl ≈ pi
```

#### Volles Vertrauen (Ω → 0)

Die Posterior-Verteilung sollte der **bedingten Verteilung** entsprechen (Szenarioanalyse):
```
X|Q → N(μ|Q, Σ|Q)  wenn Ω → 0
```

Wobei (Gleichungen 25-26):
```
μ|Q = π + ΣP'(PΣP')^(-1)(Q - Pπ)
Σ|Q = Σ - ΣP'(PΣP')^(-1)PΣ
```

**Marktformulierung:** ✓ Erreicht dies genau
**Ursprüngliche Formulierung:** Erreicht nur den bedingten Mittelwert, nicht die Kovarianz

**Test:**
```python
mu_bl, _ = bl.compute_posterior(P, Q, confidence=10000.0)  # Sehr hoch
# Für Ansichten Q = [0,10] zu Vermögenswert 0: mu_bl[0] ≈ 0,10
```

---


## Implementierungsdetails

### Kernalgorithmus

```python
def compute_posterior_market(P, Q, Omega, pi, Sigma):
    „“"
    Implementiert die Gleichungen (32) und (33) von Meucci.
    „“"
    # Zwischenberechnung: PΣP' + Ω
    M = P @ Sigma @ P.T + Omega
    M_inv = np.linalg.inv(M)
    
    # Gleichung (32): Posterior-Mittelwert
    view_adjustment = Q - P @ pi
    mu_BL = pi + Sigma @ P.T @ M_inv @ view_adjustment
    
    # Gleichung (33): Posterior-Kovarianz
    Sigma_BL = Sigma - Sigma @ P.T @ M_inv @ P @ Sigma
    
    return mu_BL, Sigma_BL
```

### Numerische Stabilität

**Herausforderungen:**
1. Matrixinversion von M = PΣP' + Ω
2. Berechnung von Sigma_BL (Differenz der Matrizen)

**Implementierte Lösungen:**

```python
# Verwendung der Pseudoinverse für nahezu singuläres M
try:
    M_inv = np.linalg.inv(M)
except np.linalg.LinAlgError:
    M_inv = np.linalg.pinv(M)  # Stabiler

# Sicherstellung der Symmetrie von Sigma_BL
Sigma_BL = (Sigma_BL + Sigma_BL.T) / 2
```

### Validierung
Validieren Sie immer die posteriore Kovarianz:

```python
validation = bl.validate_posterior(sigma_bl)

# Überprüfen:
assert validation[‚is_symmetric‘]
assert validation[‚is_psd‘]  # Positiv semidefinit
assert validation[‚condition_number‘] < 1e10  # Gut konditioniert
```

---

## Praktischer Leitfaden zur Anwendung

### Arbeitsablauf 1: Gleichgewichtsbasiert (CAPM)

```python
import numpy as np
from bl_model import BlackLittermanModel

# Schritt 1: Schätzen Sie die Kovarianz aus historischen Daten
# (unter Verwendung von exponentieller Glättung usw.)
Sigma = estimate_covariance(returns_data)

# Schritt 2: Marktkapitalisierungsgewichte abrufen
w_eq = get_market_weights()  # z. B. [0,60, 0,25, 0,10, 0,05]

# Schritt 3: Gleichgewichtsrenditen berechnen
pi = BlackLittermanModel.compute_equilibrium_returns(
    w_eq, Sigma, risk_aversion=2.5)


# Schritt 4: Modell initialisieren
bl = BlackLittermanModel(pi, Sigma, use_market_formulation=True)

# Schritt 5: Ansichten ausdrücken
P = create_view_matrix(
    n_assets=4,
    absolute_views={0: 1.0},           # US-Aktien = 10 %
    relative_views=[(1, 2, 1.0, -1.0)] # Europa – Asien = 2 %
)
Q = np.array([0.10, 0.02])

# Schritt 6: Posterior berechnen
mu_BL, Sigma_BL = bl.compute_posterior(P, Q, confidence=2.0)

# Schritt 7: Portfolio optimieren
w_optimal = mean_variance_optimize(mu_BL, Sigma_BL, risk_aversion=2.5)
```


### Workflow 2: Benutzerdefinierte Priorität

```python
# Verwenden Sie Ihr eigenes Modell für pi (z. B. Faktormodell, ML-Vorhersage)
pi = my_expected_returns_model()
Sigma = estimate_covariance(returns_data)

bl = BlackLittermanModel(pi, Sigma, use_market_formulation=True)

# Ansichten als Anpassungen Ihres Modells ausdrücken
P = create_view_matrix(n_assets=5, relative_views=[(0, 1, 1.0, -1.0)])
Q = np.array([0.03])  # „Ich denke, dass Asset 0 Asset 1 um 3 % übertreffen wird.“

mu_BL, Sigma_BL = bl.compute_posterior(P, Q, confidence=1.5)
```

### Workflow 3: Szenarioanalyse

```python
# Stresstest: Was passiert, wenn Tech um 30 % einbricht?
P = create_view_matrix(n_assets=10, absolute_views={tech_idx: 1.0})
Q = np.array([-0.30])

# Sehr hohe Konfidenz ≈ deterministisches Szenario
mu_stress, Sigma_stress = bl.compute_posterior(P, Q, confidence=10000.0)

# Analyse der Auswirkungen auf Ihr Portfolio
portfolio_return_stress = w_current @ mu_stress
```


Workflow 2: Custom Priority

```python
# Use your own model for pi (e.g., factor model, ML prediction)
pi = my_expected_returns_model()
Sigma = estimate_covariance(returns_data)

bl = BlackLittermanModel(pi, Sigma, use_market_formulation=True)

# Express views as adjustments to your model
P = create_view_matrix(n_assets=5, relative_views=[(0, 1, 1.0, -1.0)])
Q = np.array([0.03])  # “I think asset 0 will outperform asset 1 by 3%.”

mu_BL, Sigma_BL = bl.compute_posterior(P, Q, confidence=1.5)
```

### Workflow 3: Scenario analysis

```python
# Stress test: What happens if Tech slumps by 30%?
P = create_view_matrix(n_assets=10, absolute_views={tech_idx: 1.0})
Q = np.array([-0.30])

# Very high confidence ≈ deterministic scenario
mu_stress, Sigma_stress = bl.compute_posterior(P, Q, confidence=10000.0)

# Analysis of the impact on your portfolio
portfolio_return_stress = w_current @ mu_stress
```

## Fortgeschrittene Themen

### Mehrere Benutzer / Hierarchische Ansichten

Zum Kombinieren von Ansichten aus mehreren Quellen (z. B. Branchenanalysten + Makroteam):

```python
# Analyst 1: Hohes Vertrauen in Ansicht 1
P1 = np.array([[1, 0, 0, 0]])
Q1 = np.array([0,12])
relative_conf1 = np.array([1,0])  # Hohes Vertrauen

# Analyst 2: Geringe Zuversicht in Ansicht 2
P2 = np.array([[0, 1, 0, 0]])
Q2 = np.array([0,08])
relative_conf2 = np.array([3,0])  # Geringe Zuversicht (höherer Wert)

# Ansichten kombinieren
P = np.vstack([P1, P2])
Q = np.concatenate([Q1, Q2])
relative_conf = np.concatenate([relative_conf1, relative_conf2])

mu_bl, _ = bl.compute_posterior(
    P, Q,
    confidence=2.0,
    relative_confidence=relative_conf)

```

### Nichtlineare Ansichten

Für Ansichten, die hinsichtlich der erwarteten Renditen nicht linear sind (z. B. „die Volatilität wird zunehmen”), benötigen Sie Erweiterungen, die über das Standard-BL hinausgehen:

- **Meucci (2009)**: Ansichten zu Risikofaktoren mit nichtlinearer Preisbildung
- **Entropy Pooling (Meucci 2008)**: Allgemeinster Rahmen

### Stresstests für Korrelationen

Um Stresstests für Korrelationen (nicht nur Renditen) durchzuführen, verwenden Sie:

```python
# Sigma direkt für Korrelationsstress modifizieren
Sigma_stressed = stress_correlation(Sigma, asset_i, asset_j, new_corr)

# Dann BL mit gestresstem Sigma verwenden
bl_stress = BlackLittermanModel(pi, Sigma_stressed)
```

Oder verwenden Sie die Erweiterung von **Qian und Gorman (2001)** für Ansichten zu Kovarianzen.

---

## Fehlerbehebung

### Problem: Posterior-Renditen ändern sich nicht

**Symptome:** `mu_BL ≈ pi` selbst bei starken Ansichten

**Ursachen:**
1. Zu geringe Konfidenz
2. Q ≈ Pπ (Ansicht stimmt mit Prior überein)
3. Omega zu groß

**Lösungen:**
```python
# Ansicht vs. Prior überprüfen
print(„Ansicht Q:“, Q)
print(„Prior Pπ:“, P @ pi)  # Sollte unterschiedlich sein

# Konfidenz erhöhen
mu_bl, _ = bl.compute_posterior(P, Q, confidence=5.0)  # Höheren Wert versuchen

# Omega-Größe überprüfen
Omega = bl._compute_omega(P, confidence=1.0)
print(„Omega:“, Omega)  # Sollte im Vergleich zu PΣP' angemessen sein

### Problem: Posterior-Kovarianz nicht positiv definit

**Symptome:** Negative Eigenwerte in `Sigma_BL`

**Ursachen:**
1. Zu viele Ansichten (K nahe bei N)
2. Zu hohe Konfidenz der Ansichten (Ω → 0)
3. Numerische Instabilität

**Lösungen:**
```python
# Anzahl der Ansichten reduzieren
# ODER Zuverlässigkeit reduzieren
mu_bl, sigma_bl = bl.compute_posterior(P, Q, confidence=0.5)

# Eigenwerte überprüfen
eigenvalues = np.linalg.eigvalsh(sigma_bl)
print(„Min. Eigenwert:“, eigenvalues.min())

# Bei leicht negativem Wert aufgrund numerischer Werte auf PSD projizieren
if eigenvalues.min() < 0:
    eigenvalues = np.maximum(eigenvalues, 1e-8)
    Q_mat, _ = np.linalg.eigh(sigma_bl)
    sigma_bl = Q_mat @ np.diag(eigenvalues) @ Q_mat.T
```

### Problem: Extreme Allokationen

**Symptome:** Die optimalen Gewichte sind extrem (z. B. 200 % in einem Vermögenswert).

**Ursachen:**
1. Übermäßiges Selbstvertrauen
2. Unrealistische Q-Werte
3. Die Optimierung ist uneingeschränkt.

**Lösungen:**
```python
# Vertrauen reduzieren
mu_bl, sigma_bl = bl.compute_posterior(P, Q, confidence=1.0)

# Optimierung mit Einschränkungen versehen
w_opt = optimize_with_constraints(
    mu_bl, sigma_bl,
    bounds=[(0, 0.5) for _ in range(N)],  # Max. 50 % pro Vermögenswert
    budget=1.0)


# Plausibilitätsprüfung der Q-Werte
print(„View returns Q:“, Q * 100, „%“)
print(„Sind diese Werte realistisch?“)
```

### Problem: Original vs Market Give Different Results

**This is expected!** They are different models.

**Original formulation:**
- Has τ parameter (uncertainty in prior)
- Posterior covariance = (1+τ)Σ - adjustment
- More parameters to calibrate

**Market formulation:**
- No τ in posterior
- Cleaner limiting behavior
- Generally recommended

**Both are valid** - choose based on your needs.

---

## References

### Primary Reference
**Meucci, Attilio (2008).** "The Black-Litterman Approach: Original Model and Extensions"  
*SSRN Electronic Journal*. https://ssrn.com/abstract=1117574

**Key insights:**
- Market-based formulation (Section 3)
- Qualitative views (Section 2, Equation 11)
- Relationship to scenario analysis
- Extensions overview (Section 4)

### Original Papers
**Black, F., and Litterman, R. (1990).** "Asset Allocation: Combining Investor Views with Market Equilibrium"  
*Goldman Sachs Fixed Income Research*

**He, G., and Litterman, R. (2002).** "The Intuition Behind Black-Litterman Model Portfolios"

### Extensions

**Qian, E., and Gorman, S. (2001).** "Conditional Distribution in Portfolio Theory"  
*Financial Analyst Journal* 57:44-51  
→ Views on covariances

**Almgren, R., and Chriss, N. (2006).** "Optimal Portfolios from Ordering Information"  
*Journal of Risk* 9:1-47  
→ Ranking views

**Meucci, A. (2006).** "Beyond Black-Litterman: Views on Non-Normal Markets"  
*Risk* 19:114-119  
→ COP approach

**Meucci, A. (2008).** "Fully Flexible Views: Theory and Practice"  
*Risk* 21:97-102  
→ Entropy pooling (most general)

**Meucci, A. (2009).** "Enhancing the Black-Litterman Approach: Views and Stress-Test on Risk Factors"  
*Journal of Asset Management* 10:89-96  
→ Non-linear pricing, derivatives

---

In [ ]:
  """
========================================================================================
COMPLETE BLACK-LITTERMAN + FINBERT INTEGRATION - FINAL VERSION
Single-File Google Colab Ready Implementation
========================================================================================

========================================================================================
"""

# ============================================================================
# PART 1: IMPORTS & SETUP
# ============================================================================

import numpy as np
import pandas as pd
import warnings
import logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from datetime import datetime, timedelta

# Check and install dependencies for Google Colab
try:
    import yfinance as yf
except ImportError:
    print("Installing yfinance...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "yfinance", "-q"])
    import yfinance as yf

try:
    from newsapi import NewsApiClient
except ImportError:
    print("Installing newsapi-python...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "newsapi-python", "-q"])
    from newsapi import NewsApiClient

try:
    from transformers import BertTokenizer, BertForSequenceClassification
    import torch
except ImportError:
    print("Installing transformers and torch...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "torch", "-q"])
    from transformers import BertTokenizer, BertForSequenceClassification
    import torch

# Suppress warnings
warnings.filterwarnings('ignore')

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)


# ============================================================================
# PART 2: DATA STRUCTURES
# ============================================================================

@dataclass
class SentimentData:
    """Container for sentiment analysis results."""
    ticker: str
    sentiment_mean: float
    sentiment_std: float
    news_count: int
    raw_scores: List[float]


@dataclass
class BlackLittermanView:
    """Black-Litterman view specification (P, Q, Ω)."""
    P: np.ndarray
    Q: np.ndarray
    Omega: np.ndarray
    metadata: Dict


# ============================================================================
# PART 3: BLACK-LITTERMAN MODEL
# ============================================================================

class BlackLittermanModel:
    """Black-Litterman portfolio optimization (Meucci 2008)."""

    def __init__(self, pi: np.ndarray, sigma: np.ndarray):
        """Initialize Black-Litterman model."""
        self.pi = np.asarray(pi).flatten()
        self.sigma = np.asarray(sigma)

        n_assets = len(self.pi)
        if self.sigma.shape != (n_assets, n_assets):
            raise ValueError(f"Sigma shape mismatch")

        if not np.allclose(self.sigma, self.sigma.T):
            self.sigma = (self.sigma + self.sigma.T) / 2

    @staticmethod
    def compute_equilibrium_returns(w_eq: np.ndarray, sigma: np.ndarray,
                                    risk_aversion: float = 2.5) -> np.ndarray:
        """ equilibrium returns: π = 2λΣw_eq"""
        return 2 * risk_aversion * sigma @ np.asarray(w_eq).flatten()

    def compute_posterior(self, P: np.ndarray, Q: np.ndarray,
                         Omega: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Black-Litterman posterior (Meucci 2008, Eq 32-33).

        μ_BL = π + ΣP'(PΣP' + Ω)^(-1)(Q - Pπ)
        Σ_BL = Σ - ΣP'(PΣP' + Ω)^(-1)PΣ
        """
        P = np.asarray(P)
        Q = np.asarray(Q).flatten()
        Omega = np.asarray(Omega)

        K, N = P.shape
        if Omega.ndim == 0 and K == 1:
            Omega = Omega.reshape((1, 1))

        # M = PΣP' + Ω
        M = P @ self.sigma @ P.T + Omega

        # Stable inversion
        try:
            M_inv = np.linalg.inv(M)
        except:
            M_inv = np.linalg.pinv(M)

        # Posterior mean
        mu_bl = self.pi + self.sigma @ P.T @ M_inv @ (Q - P @ self.pi)

        # Posterior covariance
        sigma_bl = self.sigma - self.sigma @ P.T @ M_inv @ P @ self.sigma

        return mu_bl, sigma_bl


# ============================================================================
# PART 4: VIEW GENERATOR
# ============================================================================

class ViewGenerator:
    """ Black-Litterman views von FinBERT sentiment."""

    def __init__(self, tickers: List[str], volatilities: np.ndarray,
                 sentiment_scaling: float = 0.02,
                 base_uncertainty: float = 0.0001,
                 news_volume_weight: float = 0.5,
                 consistency_weight: float = 0.5,
                 min_news_count: int = 3):
        """Initialize ViewGenerator."""
        self.tickers = tickers
        self.volatilities = np.asarray(volatilities)
        self.N = len(tickers)
        self.sentiment_scaling = sentiment_scaling
        self.base_uncertainty = base_uncertainty
        self.news_volume_weight = news_volume_weight
        self.consistency_weight = consistency_weight
        self.min_news_count = min_news_count
        self.ticker_to_idx = {t: i for i, t in enumerate(tickers)}

    def generate_views(self, sentiment_data: Dict[str, SentimentData],
                      prior_returns: np.ndarray,
                      filter_weak_signals: bool = True,
                      min_abs_sentiment: float = 0.1) -> BlackLittermanView:
        """ Black-Litterman views from sentiment data."""

        # Filter valid views
        valid_views = []
        for ticker, data in sentiment_data.items():
            if ticker not in self.ticker_to_idx:
                continue
            if data.news_count < self.min_news_count:
                continue
            if filter_weak_signals and abs(data.sentiment_mean) < min_abs_sentiment:
                continue
            valid_views.append(data)

        if not valid_views:
            # Null view
            P = np.zeros((1, self.N))
            P[0, 0] = 1.0
            Q = np.zeros(1)
            Omega = np.eye(1) * 1e10
            metadata = {'null_view': True, 'tickers': []}
            return BlackLittermanView(P, Q, Omega, metadata)

        K = len(valid_views)

        # Build P
        P = np.zeros((K, self.N))
        for k, view_data in enumerate(valid_views):
            asset_idx = self.ticker_to_idx[view_data.ticker]
            P[k, asset_idx] = 1.0

        # Compute Q: Q = π + sentiment × σ × scaling
        Q = np.zeros(K)
        for k, view_data in enumerate(valid_views):
            asset_idx = self.ticker_to_idx[view_data.ticker]
            base_return = prior_returns[asset_idx]
            sentiment_impact = (view_data.sentiment_mean *
                              self.volatilities[asset_idx] *
                              self.sentiment_scaling)
            Q[k] = base_return + sentiment_impact

        # Compute Ω
        Omega = np.zeros((K, K))
        for k, view_data in enumerate(valid_views):
            volume_unc = 1.0 / np.sqrt(max(view_data.news_count, 1))
            consistency_unc = view_data.sentiment_std ** 2
            Omega[k, k] = (self.base_uncertainty +
                          self.news_volume_weight * volume_unc +
                          self.consistency_weight * consistency_unc)

        metadata = {
            'tickers': [v.ticker for v in valid_views],
            'sentiments': [v.sentiment_mean for v in valid_views],
            'news_counts': [v.news_count for v in valid_views]
        }

        return BlackLittermanView(P, Q, Omega, metadata)


# ============================================================================
# PART 5: FINBERT ANALYZER
# ============================================================================

class FinBERTAnalyzer:
    """FinBERT sentiment analysis."""

    def __init__(self):
        """Load FinBERT model."""
        logger.info("Loading FinBERT model...")
        self.tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")
        self.model.eval()
        logger.info("✓ FinBERT loaded")

    def analyze_sentiment(self, texts: List[str]) -> Tuple[np.ndarray, float, float]:
        """Analyze sentiment: Score = P(Pos) - P(Neg)"""
        if not texts:
            return np.array([]), 0.0, 0.5

        inputs = self.tokenizer(texts, padding=True, truncation=True,
                               max_length=512, return_tensors='pt')

        with torch.no_grad():
            outputs = self.model(**inputs)

        probs = torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()
        scores = probs[:, 0] - probs[:, 1]  # P(Pos) - P(Neg)

        return probs, float(np.mean(scores)), float(np.std(scores)) if len(scores) > 1 else 0.5


# ============================================================================
# PART 6: MAIN OPTIMIZER CLASS
# ============================================================================

class SentimentBlackLittermanOptimizer:
    """Complete integration: NewsAPI → FinBERT → Black-Litterman"""

    def __init__(self, api_key: str, tickers: List[str],
                 lookback_days: int = 252,
                 news_lookback_days: int = 7,
                 articles_per_ticker: int = 10,
                 risk_aversion: float = 2.5,
                 sentiment_scaling: float = 0.02):
        """Initialize optimizer."""
        self.api_key = api_key
        self.tickers = tickers
        self.N = len(tickers)
        self.lookback_days = lookback_days
        self.news_lookback_days = news_lookback_days
        self.articles_per_ticker = articles_per_ticker
        self.risk_aversion = risk_aversion
        self.sentiment_scaling = sentiment_scaling

        self.newsapi = NewsApiClient(api_key=api_key)
        self.finbert = FinBERTAnalyzer()

        self.sigma = None
        self.volatilities = None
        self.pi = None
        self.w_eq = None
        self.bl_model = None
        self.view_generator = None

    def fetch_news(self, ticker: str) -> List[Dict]:
        """Fetch news for ticker."""
        try:
            to_date = datetime.now()
            from_date = to_date - timedelta(days=self.news_lookback_days)

            response = self.newsapi.get_everything(
                q=ticker,
                language='en',
                sort_by='relevancy',
                page_size=self.articles_per_ticker,
                from_param=from_date.strftime('%Y-%m-%d'),
                to=to_date.strftime('%Y-%m-%d')
            )

            articles = response.get('articles', [])
            logger.info(f"  ✓ {ticker}: {len(articles)} articles")
            return articles
        except Exception as e:
            logger.error(f"  ✗ {ticker}: {e}")
            return []

    def process_ticker(self, ticker: str) -> Optional[SentimentData]:
        """Fetch news and analyze sentiment."""
        logger.info(f"\nProcessing {ticker}:")

        articles = self.fetch_news(ticker)
        if not articles:
            return None

        texts = []
        for article in articles:
            title = article.get('title', '')
            desc = article.get('description', '')
            text = f"{title}. {desc}" if desc else title
            if text:
                texts.append(text)

        if not texts:
            return None

        probs, sentiment_mean, sentiment_std = self.finbert.analyze_sentiment(texts)

        logger.info(f"  Sentiment: {sentiment_mean:+.3f} (std: {sentiment_std:.3f})")
        logger.info(f"  Sample: {texts[0][:60]}...")

        return SentimentData(
            ticker=ticker,
            sentiment_mean=sentiment_mean,
            sentiment_std=sentiment_std,
            news_count=len(texts),
            raw_scores=(probs[:, 0] - probs[:, 1]).tolist()
        )

    def load_market_data(self):
        """Load historical data - FIXED VERSION."""
        logger.info(f"\n{'='*80}")
        logger.info("LOADING MARKET DATA")
        logger.info(f"{'='*80}")

        end_date = datetime.now()
        start_date = end_date - timedelta(days=self.lookback_days + 30)

        # Download all tickers at once (more efficient)
        logger.info(f"Downloading data for {len(self.tickers)} tickers...")

        try:
            # Download all at once
            data = yf.download(self.tickers, start=start_date, end=end_date,
                             group_by='ticker', progress=False, threads=True)

            # Extract Close prices
            prices_dict = {}
            for ticker in self.tickers:
                try:
                    if len(self.tickers) == 1:
                        # Single ticker case
                        prices_dict[ticker] = data['Close']
                    else:
                        # Multiple tickers
                        prices_dict[ticker] = data[ticker]['Close']

                    logger.info(f"  ✓ {ticker}: {len(prices_dict[ticker])} days")
                except Exception as e:
                    logger.warning(f"  ⚠ {ticker}: {e}")

            if not prices_dict:
                raise ValueError("No data loaded for any ticker!")

            # Create DataFrame and align
            prices_df = pd.DataFrame(prices_dict)
            prices_df = prices_df.dropna()

            if len(prices_df) < 50:
                raise ValueError(f"Insufficient data: only {len(prices_df)} days")

            logger.info(f"\nCommon trading days: {len(prices_df)}")

            # Compute returns
            returns_df = np.log(prices_df / prices_df.shift(1)).dropna()

            # Estimate covariance
            self.sigma = returns_df.cov().values * 252
            self.sigma += np.eye(self.N) * 1e-6

            # Volatilities
            self.volatilities = np.sqrt(np.diag(self.sigma))

            logger.info("\nVolatilities (annualized):")
            for i, ticker in enumerate(self.tickers):
                logger.info(f"  {ticker}: {self.volatilities[i]*100:5.1f}%")

            # Equilibrium
            self.w_eq = np.ones(self.N) / self.N
            self.pi = BlackLittermanModel.compute_equilibrium_returns(
                self.w_eq, self.sigma, self.risk_aversion
            )

            logger.info("\nEquilibrium Returns:")
            for i, ticker in enumerate(self.tickers):
                logger.info(f"  {ticker}: {self.pi[i]*100:5.1f}%")

            # Initialize models
            self.bl_model = BlackLittermanModel(self.pi, self.sigma)
            self.view_generator = ViewGenerator(
                tickers=self.tickers,
                volatilities=self.volatilities,
                sentiment_scaling=self.sentiment_scaling
            )

        except Exception as e:
            logger.error(f"Error loading market data: {e}")
            raise

    def optimize(self) -> Optional[Dict]:
        """Run complete optimization."""
        logger.info(f"\n{'='*80}")
        logger.info("SENTIMENT BLACK-LITTERMAN OPTIMIZATION")
        logger.info(f"{'='*80}")

        # Load market data
        if self.sigma is None:
            self.load_market_data()

        # Fetch news & analyze
        logger.info(f"\n{'='*80}")
        logger.info("FETCHING NEWS & ANALYZING SENTIMENT")
        logger.info(f"{'='*80}")

        sentiment_data = {}
        for ticker in self.tickers:
            data = self.process_ticker(ticker)
            if data:
                sentiment_data[ticker] = data

        if not sentiment_data:
            logger.error("\n✗ No valid sentiment data")
            return None

        # Generate views
        logger.info(f"\n{'='*80}")
        logger.info("GENERATING VIEWS")
        logger.info(f"{'='*80}")

        view = self.view_generator.generate_views(
            sentiment_data, self.pi,
            filter_weak_signals=True, min_abs_sentiment=0.10
        )

        logger.info(f"\n✓ Generated {view.P.shape[0]} views")
        logger.info(f"  Tickers: {view.metadata.get('tickers', [])}")

        # Black-Litterman
        logger.info(f"\n{'='*80}")
        logger.info("BLACK-LITTERMAN POSTERIOR")
        logger.info(f"{'='*80}")

        mu_bl, sigma_bl = self.bl_model.compute_posterior(view.P, view.Q, view.Omega)

        logger.info("\nPosterior Returns:")
        for i, ticker in enumerate(self.tickers):
            delta = mu_bl[i] - self.pi[i]
            direction = "↑" if delta > 0 else "↓" if delta < 0 else "→"
            logger.info(f"  {ticker}: {self.pi[i]*100:5.1f}% → {mu_bl[i]*100:5.1f}% "
                       f"{direction} ({delta*100:+.1f}%)")

        # Optimize
        logger.info(f"\n{'='*80}")
        logger.info("PORTFOLIO OPTIMIZATION")
        logger.info(f"{'='*80}")

        try:
            sigma_inv = np.linalg.inv(sigma_bl)
        except:
            sigma_inv = np.linalg.pinv(sigma_bl)

        w_optimal = sigma_inv @ mu_bl / (2 * self.risk_aversion)
        w_optimal = np.maximum(w_optimal, 0)
        w_optimal = w_optimal / np.sum(w_optimal)

        logger.info("\nOptimal Weights:")
        for i, ticker in enumerate(self.tickers):
            logger.info(f"  {ticker}: {w_optimal[i]*100:5.1f}%")

        portfolio_return = w_optimal @ mu_bl
        portfolio_vol = np.sqrt(w_optimal @ sigma_bl @ w_optimal)
        sharpe = portfolio_return / portfolio_vol if portfolio_vol > 0 else 0

        logger.info(f"\nPortfolio:")
        logger.info(f"  Return: {portfolio_return*100:5.1f}%")
        logger.info(f"  Vol:    {portfolio_vol*100:5.1f}%")
        logger.info(f"  Sharpe: {sharpe:.2f}")

        return {
            'timestamp': datetime.now(),
            'tickers': self.tickers,
            'sentiment_data': sentiment_data,
            'view': view,
            'prior_returns': self.pi,
            'posterior_returns': mu_bl,
            'prior_weights': self.w_eq,
            'optimal_weights': w_optimal,
            'portfolio_return': portfolio_return,
            'portfolio_vol': portfolio_vol,
            'sharpe_ratio': sharpe
        }


# ============================================================================
# PART 7: DEMO MODE
# ============================================================================

def run_demo_mode(tickers: List[str] = None):
    """Demo mode with simulated sentiment (no API needed)."""
    if tickers is None:
        tickers = ['AAPL', 'MSFT', 'GOOGL']

    print("\n" + "="*80)
    print("DEMO MODE - SIMULATED SENTIMENT")
    print("="*80)

    # Load market data
    print("\nLoading market data...")
    end_date = datetime.now()
    start_date = end_date - timedelta(days=282)

    try:
        data = yf.download(tickers, start=start_date, end=end_date,
                          group_by='ticker', progress=False)

        prices_dict = {}
        for ticker in tickers:
            try:
                if len(tickers) == 1:
                    prices_dict[ticker] = data['Close']
                else:
                    prices_dict[ticker] = data[ticker]['Close']
                print(f"  ✓ {ticker}: {len(prices_dict[ticker])} days")
            except:
                pass

        prices_df = pd.DataFrame(prices_dict).dropna()
        returns_df = np.log(prices_df / prices_df.shift(1)).dropna()

        N = len(tickers)
        sigma = returns_df.cov().values * 252
        sigma += np.eye(N) * 1e-6
        volatilities = np.sqrt(np.diag(sigma))

        w_eq = np.ones(N) / N
        pi = BlackLittermanModel.compute_equilibrium_returns(w_eq, sigma, 2.5)

        print("\nEquilibrium Returns:")
        for i, ticker in enumerate(tickers):
            print(f"  {ticker}: {pi[i]*100:5.1f}%")

        # Simulated sentiment
        print("\n" + "="*80)
        print("SIMULATED SENTIMENT")
        print("="*80)

        np.random.seed(42)
        sentiment_data = {}

        base_sentiments = {'AAPL': 0.45, 'MSFT': 0.35, 'GOOGL': 0.25,
                          'TSLA': -0.20, 'NVDA': 0.60}

        for ticker in tickers:
            base = base_sentiments.get(ticker, np.random.uniform(-0.3, 0.3))
            news_count = np.random.randint(8, 25)
            raw_scores = np.clip(np.random.normal(base, 0.2, news_count), -1, 1)

            sentiment_data[ticker] = SentimentData(
                ticker=ticker,
                sentiment_mean=float(np.mean(raw_scores)),
                sentiment_std=float(np.std(raw_scores)),
                news_count=news_count,
                raw_scores=raw_scores.tolist()
            )

            print(f"  {ticker}: {sentiment_data[ticker].sentiment_mean:+.3f} "
                  f"({news_count} articles)")

        # Generate views
        generator = ViewGenerator(tickers, volatilities, 0.02)
        view = generator.generate_views(sentiment_data, pi, True, 0.10)

        print(f"\n✓ Generated {view.P.shape[0]} views")

        # Black-Litterman
        bl = BlackLittermanModel(pi, sigma)
        mu_bl, sigma_bl = bl.compute_posterior(view.P, view.Q, view.Omega)

        # Optimize
        sigma_inv = np.linalg.pinv(sigma_bl)
        w_optimal = sigma_inv @ mu_bl / 5.0
        w_optimal = np.maximum(w_optimal, 0)
        w_optimal = w_optimal / np.sum(w_optimal)

        # Results
        print("\n" + "="*80)
        print("FINAL PORTFOLIO")
        print("="*80)
        print(f"\n{'Ticker':<8} {'Sentiment':<12} {'Weight':<10} {'Return'}")
        print("-" * 60)

        for i, ticker in enumerate(tickers):
            sent = sentiment_data[ticker].sentiment_mean
            print(f"{ticker:<8} {sent:>+.3f}         {w_optimal[i]*100:>5.1f}%     "
                  f"{mu_bl[i]*100:>5.1f}%")

        portfolio_return = w_optimal @ mu_bl
        portfolio_vol = np.sqrt(w_optimal @ sigma_bl @ w_optimal)
        sharpe = portfolio_return / portfolio_vol if portfolio_vol > 0 else 0

        print("-" * 60)
        print(f"Portfolio Return: {portfolio_return*100:>5.1f}%")
        print(f"Portfolio Vol:    {portfolio_vol*100:>5.1f}%")
        print(f"Sharpe Ratio:     {sharpe:>5.2f}")
        print("="*80)

        return {'success': True}

    except Exception as e:
        print(f"\n✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return None


# ============================================================================
# PART 8: MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    """
    Google Colab Execution

    Two modes:
    1. DEMO MODE (simulated sentiment) - No API key needed
    2. LIVE MODE (real news) - Uses NewsAPI
    """

    # ========================================================================
    # CONFIGURATION
    # ========================================================================

    USE_DEMO_MODE = False  # Set to True for demo, False for live

    # YOUR REAL API KEY
    NEWS_API_KEY = "  "  #  https://newsapi.org/  kostenloser ApiNews mit email holen

    # Portfolio
    TICKERS = ['XLK', 'XLE', 'XLF', 'XLV', 'NVDA', 'AAPL', 'MSFT', 'GOOGL', 'XOM','CVX','GS','JPM','PFE','JNJ']
    # TICKERS = ['AAPL', 'MSFT', 'GOOGL']  # Start with 3 for testing

    # Parameters
    LOOKBACK_DAYS = 252
    NEWS_LOOKBACK_DAYS = 7
    RISK_AVERSION = 2.5

    # ========================================================================
    # RUN
    # ========================================================================

    print("\n" + "="*80)
    print("BLACK-LITTERMAN + FINBERT PORTFOLIO OPTIMIZATION")
    print("="*80)

    if USE_DEMO_MODE:
        print("\nMode: DEMO (simulated sentiment)")
        print("Set USE_DEMO_MODE = False for live news")
        print("="*80)

        results = run_demo_mode(TICKERS)

    else:
        print("\nMode: LIVE (real NewsAPI + FinBERT)")
        print(f"API Key: {NEWS_API_KEY[:10]}...")
        print(f"Tickers: {', '.join(TICKERS)}")
        print("="*80)

        try:
            optimizer = SentimentBlackLittermanOptimizer(
                api_key=NEWS_API_KEY,
                tickers=TICKERS,
                lookback_days=LOOKBACK_DAYS,
                news_lookback_days=NEWS_LOOKBACK_DAYS,
                risk_aversion=RISK_AVERSION
            )

            results = optimizer.optimize()

            if results:
                print("\n" + "="*80)
                print("✓ OPTIMIZATION COMPLETE")
                print("="*80)

                print(f"\n{'Ticker':<8} {'Sentiment':<12} {'Weight':<10} {'Return'}")
                print("-" * 60)

                for i, ticker in enumerate(TICKERS):
                    sent_data = results['sentiment_data'].get(ticker)
                    sent = sent_data.sentiment_mean if sent_data else 0.0
                    weight = results['optimal_weights'][i]
                    ret = results['posterior_returns'][i]

                    print(f"{ticker:<8} {sent:>+.3f}         {weight*100:>5.1f}%     "
                          f"{ret*100:>5.1f}%")

                print("-" * 60)
                print(f"Portfolio Return: {results['portfolio_return']*100:>5.1f}%")
                print(f"Portfolio Vol:    {results['portfolio_vol']*100:>5.1f}%")
                print(f"Sharpe Ratio:     {results['sharpe_ratio']:>5.2f}")
                print("="*80)

        except Exception as e:
            print("\n" + "="*80)
            print("✗ ERROR")
            print("="*80)
            print(f"\nError: {e}")
            import traceback
            traceback.print_exc()
            print("\nTry setting USE_DEMO_MODE = True to test without API")

    print("\n" + "="*80)
    print("DONE")
    print("="*80)


BLACK-LITTERMAN + FINBERT PORTFOLIO OPTIMIZATION

Mode: LIVE (real NewsAPI + FinBERT)
API Key: 1246c60fda...
Tickers: XLK, XLE, XLF, XLV, NVDA, AAPL, MSFT, GOOGL, XOM, CVX, GS, JPM, PFE, JNJ


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: ProsusAI/finbert
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



✓ OPTIMIZATION COMPLETE

Ticker   Sentiment    Weight     Return
------------------------------------------------------------
XLK      +0.394           7.7%       7.0%
XLE      +0.350           7.2%       5.0%
XLF      +0.163           6.7%       4.9%
XLV      -0.100           5.0%       3.6%
NVDA     +0.272           9.3%       9.0%
AAPL     +0.213           8.9%       6.1%
MSFT     -0.096           5.0%       3.8%
GOOGL    +0.479           8.9%       6.9%
XOM      -0.012           5.0%       4.0%
CVX      +0.164           7.0%       4.7%
GS       +0.149          11.4%       8.8%
JPM      +0.007           5.0%       6.7%
PFE      -0.285           6.9%       5.7%
JNJ      +0.553           5.9%       1.4%
------------------------------------------------------------
Portfolio Return:   5.9%
Portfolio Vol:      9.1%
Sharpe Ratio:      0.65

DONE
